# Question 3: Does the Complaint Type that you identified in response to question 1 have an obvious relationship with any particular characteristic or characteristics of the houses or buildings?

To analyse the characteristics of houses, we need to import a new dataset that contains the information on the houses. As we already found out, The Bronx has the highest number of heating complaints out of all the boroughs so we will be focussing on the information for houses located in the Bronx. We will also import the heating dataset we created for question 2 to find relationships between the two datasets.

## Import libraries and datasets

In [1]:
# basic libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

#import heating dataset
heating_df = pd.read_csv("heating_df.csv").drop("Unnamed: 0", axis=1)

# import housing information dataset
house_df = pd.read_csv("PLUTO_for_WEB/BX_18v1.csv")

print("Libraries and datasets imported")

Libraries and datasets imported


/Users/zenoix/opt/anaconda3/envs/IBM-DS/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (19,20,22,23,64,65,80) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [2]:
heating_df.head()

,Complaint Type,Incident Zip,Incident Address,Street Name,City,Status,Borough,Latitude,Longitude
0,HEATING,11210.0,3101 FOSTER AVENUE,FOSTER AVENUE,BROOKLYN,Closed,Unspecified,40.638696,-73.946969
1,HEATING,10466.0,1405 EAST 233 STREET,EAST 233 STREET,BRONX,Closed,Unspecified,40.888997,-73.836192
2,HEATING,11226.0,22 MARTENSE STREET,MARTENSE STREET,BROOKLYN,Closed,Unspecified,40.651236,-73.958293
3,HEATING,11212.0,1115 WILLMOHR STREET,WILLMOHR STREET,BROOKLYN,Closed,Unspecified,40.657228,-73.917447
4,HEATING,11210.0,3103 FOSTER AVENUE,FOSTER AVENUE,BROOKLYN,Closed,Unspecified,40.638696,-73.946947


In [3]:
house_df.head()

,Borough,Block,Lot,CD,CT2010,CB2010,SchoolDist,Council,ZipCode,FireComp,...,ZMCode,Sanborn,TaxMap,EDesigNum,APPBBL,APPDate,PLUTOMapID,FIRM07_FLAG,PFIRM15_FLAG,Version
0,BX,2260,1,201,19.0,1022.0,7.0,8.0,10454.0,L029,...,NaN,209S016,20901.0,E-143,0.0,NaN,1,NaN,NaN,18V1
1,BX,2260,4,201,19.0,1022.0,7.0,8.0,10454.0,L029,...,NaN,209S016,20901.0,E-143,0.0,NaN,1,NaN,NaN,18V1
2,BX,2260,10,201,19.0,1022.0,7.0,8.0,10454.0,L029,...,NaN,209S016,20901.0,E-143,0.0,NaN,1,NaN,NaN,18V1
3,BX,2260,17,201,19.0,1022.0,7.0,8.0,10454.0,L029,...,NaN,209S016,20901.0,E-143,0.0,NaN,1,NaN,NaN,18V1
4,BX,2260,18,201,19.0,1022.0,7.0,8.0,10454.0,L029,...,NaN,209S016,20901.0,E-143,0.0,NaN,1,NaN,NaN,18V1


### Isolate heating complaints only to the Bronx borough

In [12]:
bronx_heating_df = heating_df[heating_df["Borough"]=="BRONX"]
bronx_heating_df = bronx_heating_df.dropna(subset=["Street Name", "Incident Zip"])
bronx_heating_df.drop(["Status", "Latitude", "Longitude", "City", "Borough"], axis=1, inplace=True)
bronx_heating_df.reset_index(drop=True, inplace=True)
bronx_heating_df.head()

,Complaint Type,Incident Zip,Incident Address,Street Name
0,HEATING,10468.0,2781 RESERVOIR AVENUE,RESERVOIR AVENUE
1,HEATING,10453.0,10 WEST 182 STREET,WEST 182 STREET
2,HEATING,10468.0,2420 DAVIDSON AVENUE,DAVIDSON AVENUE
3,HEATING,10460.0,1536 LELAND AVENUE,LELAND AVENUE
4,HEATING,10460.0,770 GARDEN STREET,GARDEN STREET


## Very light exploration of the housing information dataframe

In [9]:
# What information are we given
house_df.columns

Index(['Borough', 'Block', 'Lot', 'CD', 'CT2010', 'CB2010', 'SchoolDist',
       'Council', 'ZipCode', 'FireComp', 'PolicePrct', 'HealthCenterDistrict',
       'HealthArea', 'SanitBoro', 'SanitDistrict', 'SanitSub', 'Address',
       'ZoneDist1', 'ZoneDist2', 'ZoneDist3', 'ZoneDist4', 'Overlay1',
       'Overlay2', 'SPDist1', 'SPDist2', 'SPDist3', 'LtdHeight', 'SplitZone',
       'BldgClass', 'LandUse', 'Easements', 'OwnerType', 'OwnerName',
       'LotArea', 'BldgArea', 'ComArea', 'ResArea', 'OfficeArea', 'RetailArea',
       'GarageArea', 'StrgeArea', 'FactryArea', 'OtherArea', 'AreaSource',
       'NumBldgs', 'NumFloors', 'UnitsRes', 'UnitsTotal', 'LotFront',
       'LotDepth', 'BldgFront', 'BldgDepth', 'Ext', 'ProxCode', 'IrrLotCode',
       'LotType', 'BsmtCode', 'AssessLand', 'AssessTot', 'ExemptLand',
       'ExemptTot', 'YearBuilt', 'YearAlter1', 'YearAlter2', 'HistDist',
       'Landmark', 'BuiltFAR', 'ResidFAR', 'CommFAR', 'FacilFAR', 'BoroCode',
       'BBL', 'CondoNo', 'Tra

In [10]:
# How many entries and columns do we have
house_df.shape

(89854, 87)

## Approach to the question

To answer question 3, I am going to take two approaches:
1. Use a binary approach where the features will either end in a complaint being made(represented by a 1) or not (represented by a 0)
2. Using total number of complaints for each street to analyse correlation

## First approach

Use the heating dataframe and add a column representing a complaint has been made.

In [ ]:
binary_heating_df = heating_df
binary_heating_df["Complaint made"] = np.ones(len(binary_heating_df), dtype=int)
binary_heating_df.drop(["Complaint Type", ])

In [ ]:
binary_heating_df.head()